# Data in a day


## Import libraries

In [ ]:
import pandas as pd
from sklearn import linear_model
%matplotlib inline

## Source the data

We need to get our data from firebase using their API - https://newton-decoded.firebaseio.com/falls.json

In [ ]:
# Read some json into dataframe using a function from pandas

## Explore and transform the data

Let's have a look at the first few rows of our data

In [ ]:
df.head()

Ideally we'd like to change labels from x and y to something more meaningful i.e. height and time^2

In [ ]:
df = df.rename(columns={'x':'height','y':'time^2'})
df.head()

Perfect! Now let's have a look at some summary stats for our data

In [ ]:
df.describe()

Now it's time to visualise the data

In [ ]:
# Using your dataframe df as a starting point, create a scatter plot of the data

## Building a model

Build a linear mathematical model - this is using calculus under the hood to find the best line.

In [ ]:
model = linear_model.LinearRegression()
model.fit(df[['height']],df[['time^2']])

## Evaluate your model

Now we've built a model we can make predictions. We'll add the results to a new column of our dataframe for convenience

In [ ]:
df['prediction'] = model.predict(df[['height']])
df.head()

Looking at tables of numbers is not great for humans. Eyeballing the model is a great to get a feel for what we have built

In [ ]:
scatter = # code to make your scatter plot goes here
# plot a line of your predictions on top of your scatter plot

We can also be quantitative when we evaluate our predictions by looking at the error between our predictions and our model.

The score gives us the R-Squared value 1 is perfect predition, 0 is terrible.

In [ ]:
model.score(df[['height']],df[['time^2']])